# 0. Import Dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv(r"C:\Users\alexs\Documents\TFM_MBD\TFM_MBD_2024_AlexSerra\preprocessed_data\other_europe.csv")

# 1. Defining train, validation, test

In [3]:
train_lim = int(df.shape[0]*0.7)
df_train = df.iloc[:train_lim]
df_test = df.iloc[train_lim:]

In [4]:
train_lim_meta = int(df_train.shape[0]*0.6)
df_train_meta = df_train.iloc[:train_lim_meta]
df_validation_meta = df_train.iloc[train_lim_meta:]

Let's define general datasets:

- df_train --> data that we are going to use to train our algorithms (prophet, autoarima, ....)
- df_test --> data that only will be used to evaluate scores of how the algorithm work.

Now, we also defined "meta" dataset. This data will be used to train the meta learner:

- df_train_meta --> son algorithm (prophet, autoarima,...) will be trained with this reduced amount of the train dataset.
- df_validation_meta --> this data will be used, after having son algorithms trained, to train the meta learner. For example, for a linear regression, it will be used to obtain the coefficients.

Once we have the metalearner trained, son algorithms will be trained with full df_train. And then using the metalearner to obtain final predictions.

# 2. Prophet

In [5]:
from prophet import Prophet

In [6]:
model_prophet = Prophet()

In [7]:
model_prophet.fit(df_train)

15:45:06 - cmdstanpy - INFO - Chain [1] start processing
15:45:07 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
forecast_prophet = model_prophet.predict(df_test[["ds"]])["yhat"].values

In [9]:
df_final_predictions= pd.DataFrame()

In [10]:
df_final_predictions["ds"] = df_test["ds"]

In [11]:
df_final_predictions ["prophet"] = forecast_prophet

In [12]:
def calculate_mape(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted)
    return np.mean(np.abs((actual - predicted) / actual)) * 100

In [13]:
mape=calculate_mape(df_test["y"],forecast_prophet)

print(f'MAPE: {mape:.2f}%')

MAPE: 21.35%


In [30]:
def prophet_predictions(df_tr, df_te):
    # df_tr should contain ds and y columns
    # df_te should contain ds column.
    m = Prophet()
    m.fit(df_tr)
    fore = m.predict(df_te[["ds"]])["yhat"].values
    return fore

# 3. Prophet adding regressors

In [14]:
model_prophet_multiva = Prophet()

In [15]:
for e in df_train.columns[2:]:
    model_prophet_multiva.add_regressor(e)

In [16]:
model_prophet_multiva.fit(df_train)

15:45:07 - cmdstanpy - INFO - Chain [1] start processing
15:45:07 - cmdstanpy - INFO - Chain [1] done processing


In [17]:
forecast_prophet_multiva = model_prophet_multiva.predict(df_test.drop(columns="y"))["yhat"].values

In [18]:
mape=calculate_mape(df_test["y"],forecast_prophet_multiva)

print(f'MAPE: {mape:.2f}%')

MAPE: 21.87%


In [19]:
df_final_predictions["prophet_multiva"] = forecast_prophet_multiva

In [41]:
def prophet_predictions_multiva(df_tr, df_te):
    # df_tr should contain ds and y columns
    # df_te should contain ds column.
    m = Prophet()
    for e in df_tr.columns[2:]:
        m.add_regressor(e)
    m.fit(df_tr)
    fore = m.predict(df_te.drop(columns="y"))["yhat"].values
    return fore

# 4. Autoarima

In [20]:
from pmdarima import auto_arima

In [21]:
df_train_arima = df_train.copy()
df_test_arima = df_test.copy()

df_train_arima.set_index('ds', inplace=True)
df_train_arima = df_train_arima[["y"]]

df_test_arima.set_index('ds', inplace=True)
df_test_arima = df_test_arima[["y"]]

In [22]:
# Train AutoARIMA model
model_autoarima = auto_arima(df_train_arima, 
                   seasonal=True,  # Change to True if you want to fit a seasonal ARIMA

                   stepwise=True,   # Set to False to perform a more exhaustive search
                   trace=True)      # Set to True to see the search progress

# Print the best model parameters
print(model_autoarima.summary())

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=268.565, Time=0.13 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=325.069, Time=0.03 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=278.007, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=292.544, Time=0.03 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=646.754, Time=0.01 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=267.768, Time=0.08 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=290.754, Time=0.04 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=265.856, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=267.825, Time=0.16 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=274.014, Time=0.07 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=272.651, Time=0.05 sec

Best model:  ARIMA(1,0,1)(0,0,0)[0] intercept
Total fit time: 0.757 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  128
Model: 

In [23]:
forecast_autoarima = model_autoarima.predict(n_periods=df_test_arima.shape[0]).values

In [24]:
df_final_predictions["autoarima"] = forecast_autoarima

In [25]:
mape=calculate_mape(df_test["y"],forecast_autoarima)

print(f'MAPE: {mape:.2f}%')

MAPE: 21.07%


In [42]:
def autoarima_predictions(df_tr, df_te):
    
    df_tr.set_index('ds', inplace=True)
    df_tr = df_tr[["y"]]
    
    df_te.set_index('ds', inplace=True)
    df_te = df_te[["y"]]
    
    m_autoarima = auto_arima(df_train_arima, 
                   seasonal=True,  # Change to True if you want to fit a seasonal ARIMA

                   stepwise=True,   # Set to False to perform a more exhaustive search
                   trace=False)      # Set to True to see the search progress
    
    fore = m_autoarima.predict(n_periods=df_te.shape[0]).values
    return fore

# 5. Trying Timeseries test split

In [26]:
df_train

ds         y   humidity        temp  gtrends_allergie  \
0    2020-01-06  3.506025  81.875606  274.946388               115   
1    2020-01-13  3.898515  77.805608  275.982323               176   
2    2020-01-20  4.174013  73.820989  275.575986               140   
3    2020-01-27  3.568473  80.112456  274.589907               150   
4    2020-02-03  3.507797  79.792778  279.608504                98   
..          ...       ...        ...         ...               ...   
123  2022-05-16  1.877216  65.656746  292.554001               130   
124  2022-05-23  2.989111  63.478175  294.610013               134   
125  2022-05-30  3.002538  64.182540  289.707639               123   
126  2022-06-06  2.186763  74.434524  292.106534               151   
127  2022-06-13  2.412533  63.375661  292.088763               112   

     gtrends_pollen allergie  gtrends_heuschnupfen  gtrends_pollen  influenza  \
0                          0                     8              22      221.0   
1                          0                    36              39        3.0   
2                          0                     5              27      530.5   
3                          0                     5              83      911.5   
4                          0                    32              10      772.0   
..                       ...                   ...             ...        ...   
123                      160                   210             441       17.0   
124                      173                   232             326        0.0   
125                       43                   179             195       10.0   
126                       11                    70             130        0.0   
127                       29                   132              99        1.0   

     pollution_Zurich_no2  pollution_Zurich_o3  pollution_Zurich_pm10  \
0               13.942857             5.657143              15.428571   
1               16.328571             5.485714              12.285714   
2               14.828571            11.000000              10.714286   
3               18.314286             2.685714              24.142857   
4                6.985714            20.357143               6.428571   
..                    ...                  ...                    ...   
123              5.242857            28.657143              13.571429   
124              5.014286            28.842857              13.428571   
125              3.385714            28.628571               7.857143   
126              4.800000            23.842857              11.714286   
127              4.000000            24.928571               8.142857   

     pollution_Zurich_pm25  pollution_Zurich_so2  
0                 3.000000              0.500000  
1                 3.714286              0.885714  
2                 2.571429              0.914286  
3                 5.000000              0.785714  
4                 1.285714              0.371429  
..                     ...                   ...  
123               1.714286              0.300000  
124               1.857143              0.314286  
125               1.000000              0.200000  
126               1.571429              0.257143  
127               1.142857              0.242857  

[128 rows x 14 columns]

In [27]:
from sklearn.model_selection import TimeSeriesSplit

In [29]:
tscv = TimeSeriesSplit()
for i, (train_index, test_index) in enumerate(tscv.split(df_train)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold 0:
  Train: index=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]
  Test:  index=[23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
Fold 1:
  Train: index=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
  Test:  index=[44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64]
Fold 2:
  Train: index=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64]
  Test:  index=[65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85]
Fold 3:
  Train: index=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85]
